# Part-3 section-1

### Contrast the effectiveness of Misclassification rate, Gini, En- tropy as impurity measures in terms of precision, recall and accuracy.


In [1]:
import numpy as np
import pandas as pd
import pprint
import sys

eps = np.finfo(float).eps
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
from numpy import log2 as log
targetattr = "left"
dataSet = pd.read_csv("./../input_data/train.csv")
numerical = ['satisfaction_level' , 'last_evaluation' , 'number_project' , 'average_montly_hours' , 'time_spend_company']
catagorical = ['Work_accident' , 'promotion_last_5years' , 'sales' , 'salary']
trainingSet , validationSet = np.split(dataSet,[int(0.8*len(dataSet))])

#### maxInfoGainNodeSplit function decides which attribute is selected as node and give the binary splite point in case of numerical attribute



In [2]:
def maxInfoGainNodeSplit(df):
    attributes = list(df)
    maxGain = 0.0
    gain = 0.0
    split = 0
    temp = 0.0
    entropybefore = entropy(df)
    for attr in attributes:
        if attr != targetattr:
            if attr in catagorical:
                gain = entropybefore - I(df,attr)
                if maxGain < gain:
                    maxGain = gain
                    node = attr
                    split = 0
            else:
                temp, ImpurityNum = InumMin(df,attr)
                gain = entropybefore - ImpurityNum
                if maxGain < gain:
                    maxGain = gain
                    node = attr
                    split = temp
    if maxGain == 0.0:
        return None , split
    else:
        return node , split


#### entropy function for entropy before spliting for missclassification rate

In [3]:
#Missclassification rate
def entropy(df):
    entropy = sys.float_info.max
    uniqueSet = df[targetattr].unique()
    for elem in uniqueSet:
        fraction = df[targetattr].value_counts()[elem]/float(len(df)+ eps)
        entropy = min(entropy, fraction) #Missclassification formula
    return entropy

#### entropy_attr function for entropy after spliting for missclassification rate


In [4]:
#Missclassification rate for optimization
def entropy_attr(num_of_ones , total):
    fraction1 = num_of_ones/(total + eps)
    fraction2 = (total - num_of_ones)/(total + eps)
    entropy = min(fraction1 , fraction2) #Missclassification formula
    return entropy

#### InumMin function to calculate minimun impurity for weighted entropy in numerical attribute

In [5]:
def InumMin(df , attribute):
    target_variables = df[targetattr].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    
    variables.sort()
    
    split = 0
    minInum = sys.float_info.max
    total_ones = df[targetattr].value_counts()[target_variables[0]]
    arr = df[[attribute, targetattr]].values
    arr = arr[np.argsort(arr[:, 0])]

    nRows = arr.shape[0]

    num_of_ones = 0
    i = 0
    for variable in variables:
        impurity = 0.0
        while (i < nRows and arr[i][0] == variable) :
            if arr[i][1] == target_variables[0]:
                num_of_ones += 1
            i += 1
        num_left = i
        fraction_left = num_left/(nRows + eps)
        if num_left == 0:
            impurity += 0.0
        else:
            impurity += fraction_left*entropy_attr(num_of_ones , num_left) 
            
        num_right = nRows - num_left
        fraction_right = num_right/(nRows + eps)
        if num_right == 0:
            impurity += 0.0
        else:
            impurity += fraction_right*entropy_attr(total_ones - num_of_ones , num_right) 
        
        if minInum > impurity:
            minInum = impurity
            split = variable
        
    return split , minInum

#### Impurity function to calculate impurity for weighted entropy

In [6]:
def I(df , attribute):
    target_variables = df[targetattr].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    den = len(df[attribute])
    impurity = 0
    for variable in variables:
        num = len(df[attribute][df[attribute] == variable])
        fraction = num/(den + eps)
        if num == 0:
            impurity += 0.0
        else:
            impurity += fraction*entropy(df[df[attribute] == variable]) 
    return impurity

#### get_subtableCat to filter data frame that has catagorical attribute

In [7]:
def get_subtableCat(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


#### get_subtableCat to filter data frame that has numerical attribute


In [8]:
def get_subtableNum(df, node, value):
    return df[df[node] <= value].reset_index(drop=True) , df[df[node] > value].reset_index(drop=True)

#### Tree Builder Function


In [9]:
def treeBuilder(df, tree=None):     
    #Here we build our decision tree   
    #Get attribute with maximum information gain
    node, split = maxInfoGainNodeSplit(df)
        
    if node is None:
        clValue,counts = np.unique(df[targetattr],return_counts=True)  
        return clValue[np.argmax(counts)]

#     print (node)
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
        
    if node in catagorical:
        #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
        attValue = np.unique(df[node])


       #We make loop to construct a tree by calling this function recursively. 
        #In this we check if the subset is pure and stops if it is pure. 

        for value in attValue:
            subtable = get_subtableCat(df,node,value)
            clValue,counts = np.unique(subtable[targetattr],return_counts=True)                        

            if len(counts)==1:#Checking purity of subset
                tree[node][value] = clValue[0]
            else:        
                tree[node][value] = treeBuilder(subtable) #Calling the function recursively 
                
    elif node in numerical:
        left_subtable , right_subtable = get_subtableNum(df,node,split)
        
        clValue,counts = np.unique(left_subtable[targetattr],return_counts=True)                      
        
        if len(counts)==1:#Checking purity of subset
            tree[node][split] = clValue[0]
        else:        
            tree[node][split] = treeBuilder(left_subtable) #Calling the function recursively 
            
        clValue,counts = np.unique(right_subtable[targetattr],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][float(split) + 0.00002] = clValue[0]
        else:        
            tree[node][float(split) + 0.00002] = treeBuilder(right_subtable) #Calling the function recursively 
            
    return tree


#### Run the code to build the tree

In [10]:
tree = treeBuilder(trainingSet)
pprint.pprint(tree)



{'satisfaction_level': {0.11: 1,
                        0.11002: {'number_project': {2: {'satisfaction_level': {0.46: {'last_evaluation': {0.57: {'last_evaluation': {0.44: 0,
                                                                                                                                      0.44002: {'average_montly_hours': {161: {'satisfaction_level': {0.31: 0,
                                                                                                                                                                                                      0.31002: {'average_montly_hours': {125: 0,
                                                                                                                                                                                                                                         125.00002: {'satisfaction_level': {0.37: {'last_evaluation': {0.51: 1,
                                                                        

#### predict function predicts the value of class attribute for a given row of input and tree

In [11]:
def predict(tree , row):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier
    for nodes in tree.keys():        
        value = row[nodes]
        if nodes in catagorical:
            if value in list((tree[nodes]).keys()):
                tree = tree[nodes][value]
            else:
                zeros = 0
                ones = 0
                for i in tree[nodes].keys():
                    if tree[nodes][i] == 0:
                        zeros += 1
                    elif tree[nodes][i] == 1:
                        ones += 1
                if zeros > ones:
                    return 0
                else:
                    return 1
        else:
            binaryValue = list(tree[nodes].keys())
            if value <= binaryValue[0]:
                tree = tree[nodes][binaryValue[0]]
            else:
                tree = tree[nodes][binaryValue[1]]
        prediction = 0
        if type(tree) is dict:
            prediction = predict(tree , row)
        else:
            prediction = tree
            break;   
            
    return prediction

#### validate function validates the given Data Frame

In [12]:
def validate(tree, df):
    true_positive = 0
    true_negative = 0 
    false_negative = 0
    false_positive = 0
    result = []
    for index, row in df.iterrows():
        value = predict(tree, row)
        result.append(value)
        if value == row[targetattr]:
            if value == 1:
                true_positive += 1
            else:
                true_negative += 1
        else:
            if value == 1:
                false_positive += 1
            else:
                false_negative += 1
    return true_positive, true_negative , false_negative, false_positive, result

In [13]:
def accuracy(true_positive , true_negative , false_negative, false_positive):
    return ((true_positive + true_negative)*100)/(true_positive + true_negative + false_positive + false_negative + eps)

In [14]:
def recall(true_positive , false_negative):
    return true_positive*100/(true_positive +  false_negative+ eps)

In [15]:
def precision(true_positive , false_positive):
    return true_positive*100/(true_positive +  false_positive + eps)

In [16]:
def f1score(recall , prescision):
    return 2/(1/(recall)+1/(prescision))

#### Run the code to calculate Accuracy, Recall, Precision, F1-score

In [17]:
def RESULT(tree, validationSet):
    true_positive , true_negative , false_negative, false_positive, result = validate(tree , validationSet)
    acc = accuracy(true_positive , true_negative , false_negative, false_positive)
    rec = recall(true_positive , false_negative)
    pre = precision(true_positive , false_positive)
    f1 = f1score(rec, pre)
    
    print("------------MISSCLASSIFICTION RATE------------")
    print("ACCURACY%: ", acc)
    print("RECALL: ", rec)
    print("PRECISION: ", pre)
    print("F1-score: ", f1)
    print(result)
    
RESULT(tree , validationSet)

------------MISSCLASSIFICTION RATE------------
ACCURACY%:  98.08718861209964
RECALL:  93.39449541284404
PRECISION:  98.64341085271317
F1-score:  95.94721960414704
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,

###### TESTING function resturns list of output


In [18]:
def TESTING(tree, df):
    result = []
    for index, row in df.iterrows():
        value = predict(tree, row)
        result.append(value)
    return result

### TEST YOUR DATASET HERE....

In [21]:
testingData = pd.read_csv("./../input_data/sample_test.csv")
res = TESTING(tree, testingData)
testingData[targetattr] = res
testingData.to_csv("./../output_data/q-1-3-1_output.csv",index = False)
print (res)
print(testingData)


[0, 1]
   satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0                0.69             0.69               3                   236   
1                0.36             0.54               2                   153   

   time_spend_company  Work_accident  promotion_last_5years        sales  \
0                   4              0                      0  product_mng   
1                   3              1                      0   accounting   

   salary  left  
0  medium     0  
1  medium     1  
